In [2]:
import argparse
import random
import numpy as np
from habitat_baselines.config.default import get_config
import habitat
import os
from habitat import Config, logger
from habitat import Config, Dataset
import matplotlib.pyplot as plt
import transformations as tf

In [3]:
import contextlib
import time
from collections import OrderedDict, defaultdict, deque
from typing import Any, Dict, List, Optional
from habitat_baselines.rl.ddppo.algo.ddp_utils import (
    EXIT,
    REQUEUE,
    add_signal_handlers,
    init_distrib_slurm,
    load_interrupted_state,
    requeue_job,
    save_interrupted_state,
)
from habitat.tasks.nav.shortest_path_follower import ShortestPathFollower
from habitat_baselines.common.baseline_registry import baseline_registry
#from habitat_baselines.common.env_utils import construct_envs
from habitat_baselines.common.environments import get_env_class
from habitat_baselines.common.rollout_storage import RolloutStorage
from habitat_baselines.common.tensorboard_utils import TensorboardWriter
from habitat_baselines.common.utils import batch_obs, linear_decay
import numpy as np
import torch
import torch.distributed as distrib
import torch.nn as nn
from gym import spaces
from gym.spaces.dict_space import Dict as SpaceDict
from torch.optim.lr_scheduler import LambdaLR
from habitat_baselines.rl.ddppo.policy.resnet_policy import PointNavResNetPolicy
from habitat_baselines.rl.ddppo.algo.ddppo import DDPPO
from habitat_baselines.common.base_trainer import BaseRLTrainer
from habitat_baselines.common.baseline_registry import baseline_registry

In [4]:
from ppotrainer import PPOTrainer
from ddpotrainer import DDPPOTrainer
from constructenv import construct_envs
from env import MyRLEnvNew
from habitat import make_dataset
from constructenv import make_env_fn
from vectorenv import VectorEnv
from utils import FrameSkip, FrameStack, draw_top_down_map

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0,3"

# Config init

In [6]:
import fileinput
filename = "/habitat-api/habitat_baselines/config/pointnav/ddppo_pointnav.yaml"
with fileinput.FileInput(filename, inplace=True, backup='.bak') as file:
    for line in file:
        print(line.replace('\"configs/tasks/pointnav_gibson.yaml\"', '\"/habitat-api/configs/tasks/pointnav_gibson.yaml\"'), end='')
with fileinput.FileInput(filename, inplace=True, backup='.bak') as file:        
    for line in file:    
        print(line.replace('GLOO', 'NCCL'), end='')

In [7]:
W = 640
H = 360
config_paths="/data/challenge_pointnav2020.local.rgbd.yaml"
config = habitat.get_config(config_paths=config_paths)
config.defrost()
config.SIMULATOR.RGB_SENSOR.HEIGHT = H
config.SIMULATOR.RGB_SENSOR.WIDTH = W
config.SIMULATOR.DEPTH_SENSOR.HEIGHT = H
config.SIMULATOR.DEPTH_SENSOR.WIDTH = W
config.DATASET.DATA_PATH = '/data/v2/{split}/{split}.json.gz'
config.TASK.MEASUREMENTS.append("TOP_DOWN_MAP")
config.TASK.SENSORS = ["HEADING_SENSOR", "COMPASS_SENSOR", "GPS_SENSOR", "POINTGOAL_SENSOR", "POINTGOAL_WITH_GPS_COMPASS_SENSOR"]
config.TASK.POINTGOAL_WITH_GPS_COMPASS_SENSOR.DIMENSIONALITY = 3
config.TASK.POINTGOAL_WITH_GPS_COMPASS_SENSOR.GOAL_FORMAT = "CARTESIAN"
config.TASK.POINTGOAL_SENSOR.DIMENSIONALITY = 3
config.TASK.POINTGOAL_SENSOR.GOAL_FORMAT = "CARTESIAN"
config.TASK.GPS_SENSOR.DIMENSIONALITY = 3
config.TASK.GPS_SENSOR.GOAL_FORMAT = "CARTESIAN"
config.TASK.AGENT_POSITION_SENSOR = habitat.Config()
config.TASK.AGENT_POSITION_SENSOR.TYPE = "position_sensor"
config.TASK.AGENT_POSITION_SENSOR.ANSWER_TO_LIFE = 42
config.TASK.SENSORS.append("AGENT_POSITION_SENSOR")
config.SIMULATOR.TURN_ANGLE = 10#0.5
config.SIMULATOR.TILT_ANGLE = 10#0.5
config.SIMULATOR.FORWARD_STEP_SIZE = 0.25#0.0125
config.ENVIRONMENT.MAX_EPISODE_STEPS = 150#*20
config.TASK.TOP_DOWN_MAP.MAX_EPISODE_STEPS = 150#*20
config.DATASET.SCENES_DIR = '/data'
config.DATASET.SPLIT = 'train'
config.SIMULATOR.SCENE = '/data/gibson/Aldrich.glb'
#config.SIMULATOR_GPU_ID = 0
config.freeze()

In [8]:
config2 = get_config('/habitat-api/habitat_baselines/config/pointnav/ddppo_pointnav.yaml', [])
ii = 0
for i in config2.TASK_CONFIG.keys():
    config2.TASK_CONFIG[i] = config[i]
    ii+=1
config = config2    

In [9]:
config.defrost()
config.TASK_CONFIG.DATASET.DATA_PATH = '/data/v2/{split}/{split}.json.gz'
config.TASK_CONFIG.DATASET.SCENES_DIR = '/data'
config.TASK_CONFIG.DATASET.SPLIT = 'train'
config.TASK_CONFIG.SIMULATOR.SCENE = '/data/gibson/Aldrich.glb'
config.TASK_CONFIG.TASK.GOAL_SENSOR_UUID = 'pos'#'pointgoal_with_gps_compass'
config.RL.PPO.num_steps = config.TASK_CONFIG.ENVIRONMENT.MAX_EPISODE_STEPS
config.RL.PPO.lr = 0.000005
config.NUM_UPDATES = 50000
config.ENV_NAME = 'MyRLEnvNew'
config.NUM_PROCESSES = 10
config.RL.FRAMESKIP = 1
config.RL.FRAMESTACK = 1
config.freeze()

## Construct one env (example)

In [10]:
print(config.ENV_NAME)
env_class = get_env_class(config.ENV_NAME)
dataset = make_dataset(config.TASK_CONFIG.DATASET.TYPE, config=config.TASK_CONFIG.DATASET)
one_env = env_class(config=config, dataset=dataset)

2020-05-24 21:59:02,433 Initializing dataset PointNav-v1


MyRLEnvNew


2020-05-24 22:00:18,248 initializing sim Sim-v0
2020-05-24 22:00:22,407 Initializing task Nav-v0


In [11]:
obs = one_env.reset()
follower = ShortestPathFollower(one_env.habitat_env.sim, 0.36, False)

NameError: name 'res' is not defined

In [39]:
for i in range(10):
    best_action = follower.get_next_action(one_env.habitat_env.current_episode.goals[0].position)
    if best_action==0 or best_action==None:
        print('DONE')
        break
    obs, reward, done, info = one_env.step(action=best_action)
    print(reward)
    print(best_action)

DONE


In [40]:
print([one_env.trux,one_env.truy])
print([one_env.goalx,one_env.goaly])

[-8.42523866293927, -7.349551993816139]
[-8.74532781409798, -7.1897156026353395]


In [13]:
observations, rewards, dones, infos = one_env.step(action=3)
top_down_map = draw_top_down_map(infos, one_env.obs["heading"][0], one_env.obs['rgb'][0].shape[0])
plt.rcParams["figure.figsize"] = [10,10]
plt.subplot(1, 2, 1)
plt.imshow(observations['depth'][:,:,0])
plt.subplot(1, 2, 2)
plt.imshow(top_down_map)
print('Env is done: ',dones)
print('Current coordinates: ',[one_env.trux,one_env.truy])
print('Goal coordinates: ',[one_env.goalx, one_env.goaly])
print('Distance to goal: ',observations['pos'])

NameError: name 'AGENT_SPRITE' is not defined

## Construct multiple envs with VectorEnv (example)

In [10]:
num_processes = config.NUM_PROCESSES
configs = [config.clone() for i in range(num_processes)]
env_class = get_env_class(config.ENV_NAME)
env_classes = [env_class for _ in range(num_processes)]

envs = VectorEnv(make_env_fn=make_env_fn,
                env_fn_args=tuple(tuple(zip(configs, env_classes, range(num_processes)))))

KeyboardInterrupt: 

In [ ]:
obs = envs.reset()

In [ ]:
outputs = envs.step([2,0,2,2])

observations, rewards, dones, infos = [list(x) for x in zip(*outputs)]

OBSERVATION_SPACE_COMMAND = 'obs'
for write_fn in envs._connection_write_fns:
    write_fn((OBSERVATION_SPACE_COMMAND, None))
obs = [read_fn() for read_fn in envs._connection_read_fns]

OBSERVATION_SPACE_COMMAND = ['trux','truy','goalx','goaly']
for i,command in enumerate(OBSERVATION_SPACE_COMMAND):
    for write_fn in envs._connection_write_fns:
        write_fn((command, None))
    OBSERVATION_SPACE_COMMAND[i] = [read_fn() for read_fn in envs._connection_read_fns]

figure, axes = plt.subplots(nrows=2, ncols=config.NUM_PROCESSES)
for i in range(config.NUM_PROCESSES):
    top_down_map = draw_top_down_map(infos[i], obs[i]["heading"][0], obs[i]['rgb'][0].shape[0])
    axes[0, i].imshow(observations[i]['depth'][:,:,0])
    axes[1, i].imshow(top_down_map)
    
print('Env is done: ',dones)
print('Current coordinates: ',list(map(list, zip(*OBSERVATION_SPACE_COMMAND[:2]))))
print('Goal coordinates: ',list(map(list, zip(*OBSERVATION_SPACE_COMMAND[2:]))))
print('Distance to goal: ',[list(observations[i]['pos']) for i in range(config.NUM_PROCESSES)])

## How to communicate with VectorEnv manually (example)

In [ ]:
data = [0,0,0,0]
STEP_COMMAND = "step"

if isinstance(data[0], (int, np.integer, str)):
    data = [{"action": {"action": action}} for action in data]
envs._is_waiting = True
for write_fn, args in zip(envs._connection_write_fns, data):
    write_fn((STEP_COMMAND, args))

outputs = []
for read_fn in envs._connection_read_fns:
    outputs.append(read_fn())
observations, rewards, dones, infos = [list(x) for x in zip(*outputs)]    
envs._is_waiting = False
plt.imshow(observations[1]['depth'][:,:,0])

In [ ]:
OBSERVATION_SPACE_COMMAND = 'obs'
for write_fn in trainer.envs._connection_write_fns:
    write_fn((OBSERVATION_SPACE_COMMAND, None))
obs = [read_fn() for read_fn in trainer.envs._connection_read_fns]

In [ ]:
outputs = trainer.envs.step([3 for i in range(config.NUM_PROCESSES)])
observations, rewards, dones, infos = [list(x) for x in zip(*outputs)]

# Run training process

In [10]:
random.seed(config.TASK_CONFIG.SEED)
np.random.seed(config.TASK_CONFIG.SEED)

In [11]:
from arguments import get_args
argumnts = ''
args = get_args(argumnts)
args.num_processes = config.NUM_PROCESSES
config.defrost()
config.RL.PPO.num_steps = config.TASK_CONFIG.ENVIRONMENT.MAX_EPISODE_STEPS
config.RL.DDPPO.pretrained_weights = 'data/new_checkpoints/ckpt.94.pth'
config.RL.DDPPO.pretrained_encoder = False
config.RL.DDPPO.pretrained = False
config.freeze()

Auto GPU config:
Number of processes: 24
Number of processes on GPU 0: 8
Number of processes per GPU: 16


In [12]:
trainer_init = baseline_registry.get_trainer(config.TRAINER_NAME)

In [13]:
trainer = trainer_init(config)

2020-05-28 18:46:45,429 config: BASE_TASK_CONFIG_PATH: /habitat-api/configs/tasks/pointnav_gibson.yaml
CHECKPOINT_FOLDER: data/new_checkpoints
CHECKPOINT_INTERVAL: 50
CMD_TRAILING_OPTS: []
ENV_NAME: MyRLEnvNew
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/new_checkpoints
LOG_FILE: train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 10
NUM_UPDATES: 50000
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
  PLANNER_MAX_STEPS: 500
  PREPROCESS_MAP: True
  SLAM_SETTINGS_PATH: habitat_baselines/slambased/data/mp3d3_small1k.yaml
  SLAM_VOCAB_PATH: habitat_baselines/slambased/data/ORBvoc.txt
RL:
  DDPPO:
    backbone: resnet50
    distrib_backend: NCCL
    num_recurrent_layers: 2
    pretrained: Fal

In [14]:
trainer.train_init(args)

2020-05-28 18:46:50,800 Initializing dataset PointNav-v1


0
1
1
1
1
1
1
1
1
1
1


2020-05-28 18:47:11,422 agent number of trainable parameters: 12454917


In [15]:
pretrained_state = torch.load(config.RL.DDPPO.pretrained_weights, map_location="cpu")
trainer.actor_critic.load_state_dict({k[len("actor_critic.") :]: v for k, v in pretrained_state["state_dict"].items()})

<All keys matched successfully>

In [ ]:
trainer.train()

RESET
{'distance_to_goal': 2.063289451599121, 'softspl': 0.6022128105163574, 'spl': 0.0, 'success': 0.0, 'top_down_map.agent_angle': -1.3942545890808105}
{'distance_to_goal': 3.1058331298828126, 'softspl': 0.44567634582519533, 'spl': 0.06314438819885254, 'success': 0.08, 'top_down_map.agent_angle': -1.5142601013183594}
{'distance_to_goal': 2.8387554931640624, 'softspl': 0.4567730712890625, 'spl': 0.04490628719329834, 'success': 0.06, 'top_down_map.agent_angle': -1.4932855224609376}
{'distance_to_goal': 2.8724660237630206, 'softspl': 0.46991312662760415, 'spl': 0.04981921831766764, 'success': 0.06666666666666667, 'top_down_map.agent_angle': -1.5522231038411458}
{'distance_to_goal': 2.9186529541015624, 'softspl': 0.4684936141967773, 'spl': 0.03736441373825073, 'success': 0.05, 'top_down_map.agent_angle': -1.528429718017578}
{'distance_to_goal': 2.8378771134005247, 'softspl': 0.48035955065079317, 'spl': 0.041117602632245946, 'success': 0.05343511450381679, 'top_down_map.agent_angle': -1.5

2020-05-28 18:53:20,128 update: 10	fps: 46.551	
2020-05-28 18:53:20,130 update: 10	env-time: 293.686s	pth-time: 60.128s	frames: 16500.0
2020-05-28 18:53:20,131 Average window size: 11  distance_to_goal: 2.708  reward: 3.513  softspl: 0.492  spl: 0.034  success: 0.046  top_down_map.agent_angle: -1.728


{'distance_to_goal': 2.7078445843809864, 'softspl': 0.49220538961476296, 'spl': 0.03408396929159932, 'success': 0.04597701149425287, 'top_down_map.agent_angle': -1.7279784688547652}
{'distance_to_goal': 2.7731493023079885, 'softspl': 0.48904214778416594, 'spl': 0.03404988705272406, 'success': 0.045774647887323945, 'top_down_map.agent_angle': -1.6778301185285542}
{'distance_to_goal': 2.7598691878780244, 'softspl': 0.48821854129914316, 'spl': 0.031194090074108494, 'success': 0.041935483870967745, 'top_down_map.agent_angle': -1.6999214418472783}
{'distance_to_goal': 2.7344513831779973, 'softspl': 0.4869408412286413, 'spl': 0.03414624476293374, 'success': 0.04678362573099415, 'top_down_map.agent_angle': -1.6699093824241593}
{'distance_to_goal': 2.741518601127293, 'softspl': 0.48624262602432916, 'spl': 0.0317337383394656, 'success': 0.043478260869565216, 'top_down_map.agent_angle': -1.6920637047809104}
{'distance_to_goal': 2.7455004820690205, 'softspl': 0.4890918683156409, 'spl': 0.03185345

2020-05-28 18:58:29,605 update: 20	fps: 47.445	
2020-05-28 18:58:29,608 update: 20	env-time: 548.907s	pth-time: 113.934s	frames: 31500.0
2020-05-28 18:58:29,609 Average window size: 21  distance_to_goal: 2.815  reward: 3.354  softspl: 0.484  spl: 0.030  success: 0.040  top_down_map.agent_angle: -1.647


{'distance_to_goal': 2.8146984281994047, 'softspl': 0.4839163353329613, 'spl': 0.02972510746547154, 'success': 0.04, 'top_down_map.agent_angle': -1.6472735305059525}
{'distance_to_goal': 2.8039470880681816, 'softspl': 0.4876619096235795, 'spl': 0.029794776222922585, 'success': 0.04, 'top_down_map.agent_angle': -1.632542169744318}
{'distance_to_goal': 2.789397400008124, 'softspl': 0.4873910570062013, 'spl': 0.02840056659030749, 'success': 0.038128249566724434, 'top_down_map.agent_angle': -1.6328811513282604}
{'distance_to_goal': 2.7988385996802805, 'softspl': 0.4855207688737624, 'spl': 0.02952163054211305, 'success': 0.039603960396039604, 'top_down_map.agent_angle': -1.6395240506716686}
{'distance_to_goal': 2.8086190125765205, 'softspl': 0.48619674995988843, 'spl': 0.028262414073492115, 'success': 0.037914691943127965, 'top_down_map.agent_angle': -1.6421287982573065}
{'distance_to_goal': 2.7716505160144447, 'softspl': 0.4875518706630004, 'spl': 0.02927658276975695, 'success': 0.03927492

...

{'distance_to_goal': 3.9135223084509967, 'softspl': 0.5013512899709303, 'spl': 0.02429325002372463, 'success': 0.03322259136212625, 'top_down_map.agent_angle': -1.6408740072155317}
{'distance_to_goal': 3.892771656042703, 'softspl': 0.5034803197553898, 'spl': 0.02359716374284988, 'success': 0.03233830845771144, 'top_down_map.agent_angle': -1.6231607465601678}
{'distance_to_goal': 3.8538994761560694, 'softspl': 0.5060970643450661, 'spl': 0.02539207086791488, 'success': 0.03468208092485549, 'top_down_map.agent_angle': -1.6208575726736685}
{'distance_to_goal': 3.870186826952803, 'softspl': 0.5048000904910346, 'spl': 0.026795795311735173, 'success': 0.036273701566364384, 'top_down_map.agent_angle': -1.6171219865841406}
{'distance_to_goal': 3.8792769746302382, 'softspl': 0.5032114422311524, 'spl': 0.027152281661648778, 'success': 0.03697617091207888, 'top_down_map.agent_angle': -1.6177476994209634}
{'distance_to_goal': 3.891608760749386, 'softspl': 0.5022083129382934, 'spl': 0.02834686229309

2020-05-28 22:19:02,179 update: 430	fps: 50.920	
2020-05-28 22:19:02,181 update: 430	env-time: 10328.698s	pth-time: 2346.364s	frames: 646500.0
2020-05-28 22:19:02,182 Average window size: 50  distance_to_goal: 3.861  reward: 4.026  softspl: 0.501  spl: 0.029  success: 0.039  top_down_map.agent_angle: -1.627


{'distance_to_goal': 3.8606728626418154, 'softspl': 0.5010333439905794, 'spl': 0.028604869131524133, 'success': 0.03889789303079417, 'top_down_map.agent_angle': -1.627223776843598}
{'distance_to_goal': 3.8337661310728746, 'softspl': 0.5018159318066802, 'spl': 0.028581707294170672, 'success': 0.038866396761133605, 'top_down_map.agent_angle': -1.6392642372532895}
{'distance_to_goal': 3.828185096153846, 'softspl': 0.5019909816232287, 'spl': 0.03039340123473874, 'success': 0.04129554655870445, 'top_down_map.agent_angle': -1.6371540517459515}
{'distance_to_goal': 3.8620711959814216, 'softspl': 0.5001311417735258, 'spl': 0.03090781734139931, 'success': 0.0420032310177706, 'top_down_map.agent_angle': -1.6392192193495052}
{'distance_to_goal': 3.857184439367932, 'softspl': 0.5022006181214661, 'spl': 0.03292058473257333, 'success': 0.0444264943457189, 'top_down_map.agent_angle': -1.6579684502473748}
{'distance_to_goal': 3.865452350674859, 'softspl': 0.5018428584244057, 'spl': 0.03284100233595953

2020-05-28 22:23:46,975 update: 440	fps: 50.958	
2020-05-28 22:23:46,977 update: 440	env-time: 10560.119s	pth-time: 2399.319s	frames: 661500.0
2020-05-28 22:23:46,978 Average window size: 50  distance_to_goal: 3.770  reward: 3.924  softspl: 0.502  spl: 0.035  success: 0.047  top_down_map.agent_angle: -1.623


{'distance_to_goal': 3.7698460578162174, 'softspl': 0.5020772282043021, 'spl': 0.03473065563083924, 'success': 0.046788263283108644, 'top_down_map.agent_angle': -1.6230932442939632}
{'distance_to_goal': 3.7169471153846154, 'softspl': 0.5049198885742961, 'spl': 0.03515652226229493, 'success': 0.047581284694686754, 'top_down_map.agent_angle': -1.625294866115682}
{'distance_to_goal': 3.711275866959353, 'softspl': 0.504686170425957, 'spl': 0.03381421722195331, 'success': 0.04577742699289661, 'top_down_map.agent_angle': -1.6140167549452447}
{'distance_to_goal': 3.7286792818154995, 'softspl': 0.5040784065972905, 'spl': 0.03251987700383556, 'success': 0.044059795436664044, 'top_down_map.agent_angle': -1.6029010475572876}
{'distance_to_goal': 3.7497596860305644, 'softspl': 0.5016902337627351, 'spl': 0.03247510975804822, 'success': 0.0438871473354232, 'top_down_map.agent_angle': -1.5994300961868142}
{'distance_to_goal': 3.758104249951248, 'softspl': 0.49933404074444226, 'spl': 0.032343574321586

In [ ]:
pretrained_state = torch.load(config.RL.DDPPO.pretrained_weights, map_location="cpu")
trainer.actor_critic.net.visual_encoder.backbone.load_state_dict({k[len("actor_critic.net.visual_encoder.backbone.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.visual_encoder.backbone" in k})
for param in trainer.actor_critic.net.visual_encoder.backbone.parameters():
    param.requires_grad = False
trainer.actor_critic.net.prev_action_embedding.load_state_dict({k[len("actor_critic.net.prev_action_embedding.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.prev_action_embedding" in k})
for param in trainer.actor_critic.net.prev_action_embedding.parameters():
    param.requires_grad = False
trainer.actor_critic.net.tgt_embeding.load_state_dict({k[len("actor_critic.net.tgt_embeding.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.tgt_embeding" in k})
for param in trainer.actor_critic.net.tgt_embeding.parameters():
    param.requires_grad = False
trainer.actor_critic.net.visual_encoder.compression.load_state_dict({k[len("actor_critic.net.visual_encoder.compression.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.visual_encoder.compression" in k})
for param in trainer.actor_critic.net.visual_encoder.compression.parameters():
    param.requires_grad = False
trainer.actor_critic.net.visual_fc.load_state_dict({k[len("actor_critic.net.visual_fc.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.visual_fc" in k})
for param in trainer.actor_critic.net.visual_fc.parameters():
    param.requires_grad = False
trainer.actor_critic.net.state_encoder.load_state_dict({k[len("actor_critic.net.state_encoder.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.state_encoder" in k})
#for param in trainer.actor_critic.net.state_encoder.parameters():
#    param.requires_grad = False
trainer.actor_critic.critic.load_state_dict({k[len("actor_critic.critic.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.critic" in k})
#for param in trainer.actor_critic.critic.parameters():
#    param.requires_grad = False
trainer.actor_critic.action_distribution.load_state_dict({k[len("actor_critic.action_distribution.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.action_distribution" in k})
#for param in trainer.actor_critic.action_distribution.parameters():
#    param.requires_grad = False